In [5]:
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, get_double_side_view, show_selected_joints, get_fixed_boundary_joint)
import vis.fields
import matplotlib.cm as cm
import time

In [6]:
rod_length = 0.09982061776030171
width = rod_length / 15 * 5
thickness = width / 5 * 0.35
print(width, thickness)
io = InputOrganizer('bob_1', thickness, width, weaving_dir)
# io = InputOrganizer('bunny_quad_1', thickness, thickness, weaving_dir)

#  [ ear 214, ear 255, nose 39]

0.033273539253433904 0.0023291477477403733


In [7]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 0.01
sw = 0.01

In [8]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, interleaving_type=InterleavingType.weaving)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = get_double_side_view(curved_linkage, flip = True)
curved_linkage_view.show()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='white', intensity=0.6, position=(0.0…

In [9]:
curved_linkage.setExtendedDoFsPSRL(np.load('bob_1_dof.npy'))

In [10]:
topColor =[180/255., 158/255., 119/255.]
bottomColor =[79/255., 158/255., 246/255.]
heights = curved_linkage.visualizationGeometryHeightColors()
colors = np.take(np.array([bottomColor, topColor]), heights < heights.mean(), axis=0)

In [11]:
curved_linkage.attraction_weight = 1e-5

In [12]:
def eqm_callback(prob, i):
    curved_linkage_view.update(scalarField = colors)

In [13]:
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS, fixedVars = [])

0	0.0225448	5.75435	5.75435	0.25	1
1	0.0170254	4.05216	4.05216	0.5	1
2	0.0127271	26.4252	26.4252	0.5	1
3	0.00916852	6.36665	6.36665	1	1
4	0.00797802	2.67997	2.67997	1	1
5	0.00707711	1.5477	1.5477	1	1
6	0.00596226	1.20414	1.20414	1	1
7	0.00457222	0.895551	0.895551	1	1
8	0.00312949	0.614138	0.614138	1	1
9	0.00193187	0.392489	0.392489	1	1
10	0.00115204	0.247098	0.247098	1	1
11	0.000733415	0.158703	0.158703	1	1
12	0.000515929	0.0989867	0.0989867	1	1
13	0.000384608	0.0584237	0.0584237	1	1
14	0.000290013	0.0331003	0.0331003	1	1
15	0.000215548	0.0182154	0.0182154	1	1
16	0.000155964	0.0101343	0.0101343	1	1
17	0.000109915	0.00542953	0.00542953	1	1
18	7.70121e-05	0.00456907	0.00456907	1	1
19	5.65131e-05	0.00595544	0.00595544	1	1
20	4.58938e-05	0.00774052	0.00774052	1	1
21	4.11793e-05	0.00794392	0.00794392	1	1
22	3.91617e-05	0.00524756	0.00524756	1	1
23	3.82525e-05	0.00312274	0.00312274	1	1
24	3.78474e-05	0.0016782	0.0016782	1	1
25	3.76927e-05	0.000784607	0.000784607	1	1
26	3.76437e-05	0.00027644

In [14]:
def visualize_crossing(index, linkage, view):
    vf = show_selected_joints(linkage, [index], flip = False)
    view.update(vectorField = vf, scalarField = colors)

In [18]:
ribbons = order_segments_by_ribbons(curved_linkage)
_, _, _, _, all_joint_index, _ = get_turning_angle_and_length_from_ordered_rods(ribbons, curved_linkage, rest = True)

def get_ribbon_crossing_list(index, linkage, view):
    selected_list = []
    selected_ribbon = []
    for ribbon_index, index_list in enumerate(all_joint_index):
        if index in set(index_list):
            selected_ribbon.append(ribbon_index)
            selected_list.append(index_list)
    print("The crossing {} belongs to ribbon {}".format(index, ', '.join([str(x) for x in selected_ribbon])))
    for i in range(len(selected_list)):
        print('Ribbon {}: {}'.format(selected_ribbon[i], selected_list[i]))
    return [selected_ribbon, selected_list]
    

In [19]:
def select_crossing(crossing_index):
    visualize_crossing(crossing_index, curved_linkage, curved_linkage_view)
    return get_ribbon_crossing_list(crossing_index, curved_linkage, curved_linkage_view)

vNum = [279]
[selected_ribbon, selected_list] = select_crossing(vNum[0])

vf = show_selected_joints(curved_linkage, vNum)
curved_linkage_view.update(vectorField = vf, scalarField = colors)
curved_linkage_view.show()

The crossing 279 belongs to ribbon 0, 2
Ribbon 0: [2, 0, 8, 18, 34, 52, 72, 94, 115, 127, 126, 151, 184, 216, 252, 277, 275, 276, 273, 274, 260, 258, 221, 222, 230, 231, 233, 232, 225, 190, 168, 137, 138, 136, 135, 164, 194, 230, 257, 282, 283, 284, 274, 246, 210, 179, 150, 125, 124, 119, 118, 117, 116, 121, 120, 145, 170, 200, 234, 266, 279, 288, 272, 244, 208, 202, 171, 172, 173, 174, 175, 176, 182, 183, 214, 247, 276, 290, 298, 299, 293, 286, 265, 233, 195, 196, 166, 138, 111, 112, 88, 68, 47, 48, 31, 32, 18, 19, 20, 21, 36, 54, 74, 97, 119, 144, 176, 206, 241, 253, 215, 184, 153, 128, 123, 100, 99, 124, 144, 175, 205, 239, 270, 278, 289, 272, 243, 207, 201, 170, 140, 114, 95, 72, 70, 49, 48, 29, 30, 27, 28, 45, 65, 87, 110, 135, 165, 195, 232, 229, 193, 192, 159, 160, 155, 156, 162, 163, 164, 165, 196, 197, 167, 168, 158, 157, 132, 133, 160, 189, 224, 260, 284, 294, 299, 296, 285, 262, 227, 226, 248, 212, 211, 210, 209, 214, 213, 249, 253, 217, 185, 186, 169, 139, 114, 92, 71, 50, 

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(PointLight(color='white', intensity=0.6, position=(0.0…